dnn-hy 实验四 by 孟
github源码：http://nbviewer.jupyter.org/github/cavaunpeu/flight-delays/blob/master/notebooks/flight-prediction.ipynb


参照如上 github 程序来进行的实验四- Transfer Learning for Flight Delay Prediction via Variational Autoencoders

迁移学习 训练两个网络，一个用于表达机场位置等的隐层信息，也就是编码网络 Variational Autoencoders，另一个网络用编码以及其它天气等特征进行预测。

编码网络：将 origin_airport 和 dest_airport 单独挑选出来，并且加入各个机场经纬度的特征，并制造负样本。 label 为航线是否存在。通过变分自编码器来构成编码网络，学习机场位置间的潜在信息，将隐层单元提取出来作为机场潜在信息的表示输入到预测网络。

负样本：由于数据给的均是正样本，制造负样本首先先将正样本标签去掉，然后按照机场出现的频率来采样。

变分自编码器：讲下思想，之前比如学习得到 AMC [1，1.3，0.22，4.5]，某个精确的值来表示 AMC， 如果有干扰，当[1.2, 1.3, 0.22, 4.5] 仅变换第一个数值，就可能表示 TEW 机场了，而通过变分自编码器，从高斯分布里采样能够使得 [1.2-1.4, 1.3-1.5, 0.1-0.3, 4.0-4.7] 的数值均表示为 AMC，变分自编码器是一种生成式模型，能够形成对机场名称形成新的表示。


预测网络：将编码网络的隐层单元信息提取出来 concat 天气等信息作为特征进行预测。（只是简单结构的 DNN ）

结构太复杂，第一步编码网络训练起来就很困难，尝试调参后，效果仍不好，所以放弃。训练两个网络，且并非 end-to-end 模型还是有点扯。

In [1]:
import keras
import keras.backend as K

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
from __future__ import print_function

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pydot

import seaborn as sns
from abc import ABCMeta, abstractmethod


from IPython.display import SVG
from sklearn import preprocessing
from sklearn.utils import shuffle 
from sklearn.model_selection import KFold, train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

#from keras.utils.np_utils import to_categorical
from keras import initializers
from keras.callbacks import Callback
from keras.layers import Input, Dense, Dropout, Embedding, dot, Flatten, concatenate, BatchNormalization, Layer as KerasLayer
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras.utils.vis_utils import model_to_dot
from keras.regularizers import l2
from keras.losses import binary_crossentropy

%matplotlib inline

In [3]:
data = pd.read_feather('/home/ubuntu/data/flight_data_7_18.feather')
airport_points = pd.read_feather('/home/ubuntu/data/airport_points.feather')

In [4]:
airport_points.head()

,airport_id,lat,lon
0,AAT,47.750361,88.0844
1,ACX,25.081176,104.9590
2,AEB,23.719444,106.9590
3,AHJ,32.515833,102.3590
4,AKU,41.262500,80.2917


In [5]:
data['feat1'] = data['delay_cat'].map(lambda x: 1 if x == '[-500,30)' or x == '[30,60)' or x == '[60,90)' or x == '[90,120)' else 0)
data['feat2'] = data['feat1'].map(lambda x: 1-x)
data = data.drop(['delay_cat'], axis=1)

In [6]:
data = data.dropna()
#data = shuffle(data)
data = data.sample(frac=0.5)
#print(data.info())

In [7]:
data['feat1'].value_counts()

1    3296583
0     226319
Name: feat1, dtype: int64

In [8]:
from sklearn.utils import resample

data_majority = data[data.feat1==1]
data_minority = data[data.feat1==0]
data_majority_downsampled = resample(data_majority,
                                     replace=False,    # sample without replacement
                                     n_samples=226020,     # to match minority class
                                     random_state=1) # reproducible results
data_downsampled = pd.concat([data_majority_downsampled, data_minority])
data = data_downsampled

In [9]:
data['sch_time_year'] = data['sch_dep_time'].map(lambda x: x.year)
data['sch_time_month'] = data['sch_dep_time'].map(lambda x: x.month)
data['sch_time_day'] = data['sch_dep_time'].map(lambda x: x.day)
data['sch_time_hour'] = data['sch_dep_time'].map(lambda x: x.hour)

In [10]:
data['sch_time_year'] = (data['sch_time_year'].map(lambda x: int(str(x)[2:])))

In [11]:
month_num = {1:6, 2:2, 3:2, 4:5, 5:0, 6:3, 7:5, 8:1, 9:4, 10:6, 11:2, 12:4}
data['year_num'] = data['sch_time_year'].map(lambda x: (int(x / 4) + x) % 7)
data['month_num']=  data['sch_time_month'].map(lambda x: month_num[x])
data['day_num'] = data['sch_time_day']
data['sch_time_day'] = data.apply(lambda x: (x['year_num'] + x['month_num'] + x['day_num'])%7, axis=1)
time_drop = ['year_num', 'month_num', 'day_num']
data = data.drop(time_drop,axis=1)
data = shuffle(data)

In [12]:
data['sch_time_hour'] = data['sch_time_hour'].map(lambda x: 24 if x==0 else x)
for i in range(8):  #每3个小时一个区间
    data['sch_time_hour'] = data['sch_time_hour'].map(lambda x: 0+i if x>0+3*i and x<=3+3*i else x)

In [13]:
routes_lab = ['origin_airport', 'dest_airport']

In [14]:
routes = data[routes_lab]
routes['exists'] = 1

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [15]:
routes.head()

,origin_airport,dest_airport,exists
3460511,CAN,WEF,1
671769,PVG,HDG,1
1430734,HET,CIF,1
1931332,HAK,CAN,1
4343447,SJW,KWL,1


In [16]:
routes_gy = routes.groupby(['origin_airport', 'dest_airport'])

In [17]:
routes_gy_num = routes_gy.count()

In [18]:
print('There are {} unique routes.'.format(routes_gy_num.shape[0] ))

There are 5082 unique routes.


In [19]:
routes_gy_num.sort_values(by='exists', ascending=False).head(20).T

origin_airport   SZX   SHA         CAN   SHA   PEK         SZX   HGH   CAN  \
dest_airport     SHA   SZX   CAN   SHA   PEK   SHA   SZX   PEK   CAN   PEK   
exists          2873  2564  2509  2487  2294  2276  2111  1989  1805  1721   

origin_airport   PEK   CTU   XMN   CAN   PEK   SHA   HGH   JHG   SZX   CAN  
dest_airport     CTU   PEK   SHA   HGH   CAN   XMN   SZX   KMG   HGH   CTU  
exists          1680  1644  1530  1512  1485  1423  1403  1338  1291  1287

In [20]:
# compute airport frequencies for negative sampling
all_airports = routes['origin_airport'].tolist() + routes['dest_airport'].tolist()
airport_counts = pd.Series(all_airports).value_counts()
airport_probs = (airport_counts ** .75) / (airport_counts ** .75).sum()

In [21]:
def compute_unique_negative_dests(airport, routes=routes):
    return routes[ routes['origin_airport'] != airport ]['dest_airport'].unique()


def draw_negative_samples(n, neg_dest_probs):
    samples_mode = np.inf
    while samples_mode >= 0.75*n:
        negative_sample_idxs = np.random.multinomial(n, neg_dest_probs)
        samples_mode = negative_sample_idxs.max() if n >= 4 else -np.inf
    negative_samples = []
    for dest, count in zip(neg_dest_probs.index, negative_sample_idxs):
        if count > 0:
            negative_samples += count*[dest]
    return negative_samples

In [22]:
# append `routes` with negative samples
negative_sample_dfs = []

for i, airport in enumerate(set(routes['origin_airport'])):
    n_routes = len(routes[ routes['origin_airport'] == airport ])
    negative_dests = compute_unique_negative_dests(airport)
    negative_dest_probs = airport_probs[negative_dests] / airport_probs[negative_dests].sum()
    negative_samples = draw_negative_samples(n_routes, negative_dest_probs)
    df = pd.DataFrame({
        'origin_airport': airport,
        'dest_airport': negative_samples,
        'exists': 0
    })
    negative_sample_dfs.append(df)
    

negative_routes = pd.concat(negative_sample_dfs, axis=0)
routes = pd.concat([routes, negative_routes])

In [23]:
routes_index_num = len(routes.index)
routes['index'] = list(range(routes_index_num))

In [24]:
routes = routes.set_index('index')
routes = shuffle(routes)

In [25]:
dic_airport_lat = {}
dic_airport_lat = dict(zip(airport_points['airport_id'], airport_points['lat']))
dic_airport_lon = {}
dic_airport_lon = dict(zip(airport_points['airport_id'], airport_points['lon']))  

In [26]:
routes.head()

,dest_airport,exists,origin_airport
index,,,
45689,SHE,1,PVG
131899,URC,1,YIN
900326,FOC,0,YIC
185477,SHA,1,CKG
354011,CAN,1,HAK


In [27]:
geo_lab = ['airport_points_lat_ori', 'airport_points_lon_ori', 'airport_points_lat_dest', 'airport_points_lon_dest']

In [28]:
routes['airport_points_lat_ori'] = routes['origin_airport'].map(lambda x: dic_airport_lat[x])
routes['airport_points_lat_dest'] = routes['dest_airport'].map(lambda x: dic_airport_lat[x])
routes['airport_points_lon_ori'] = routes['origin_airport'].map(lambda x: dic_airport_lon[x])
routes['airport_points_lon_dest'] = routes['dest_airport'].map(lambda x: dic_airport_lon[x])

In [29]:
routes.head()

,dest_airport,exists,origin_airport,airport_points_lat_ori,airport_points_lat_dest,airport_points_lon_ori,airport_points_lon_dest
index,,,,,,,
45689,SHE,1,PVG,31.143333,41.640000,121.8050,123.4840
131899,URC,1,YIN,43.955833,43.907489,81.3303,87.4791
900326,FOC,0,YIC,27.803333,25.935000,114.3080,119.6630
185477,SHA,1,CKG,29.719167,31.198056,106.6420,121.3360
354011,CAN,1,HAK,19.934722,23.392500,110.4590,113.2990


In [30]:
all_airports = routes['origin_airport'].tolist() + routes['dest_airport'].tolist()
unique_airports = set(all_airports)
airport_to_id = {airport: index for index, airport in enumerate(unique_airports)}

routes['origin_id'] = routes['origin_airport'].map(airport_to_id)
routes['dest_id'] = routes['dest_airport'].map(airport_to_id)

In [31]:
routes = routes.drop(['origin_airport', 'dest_airport'], axis=1)

In [32]:
# build X_routes, y_routes
geo_cols = ['airport_points_lat_ori', 'airport_points_lon_ori', 'airport_points_lat_dest', \
            'airport_points_lon_dest']
X_r = routes[ ['origin_id', 'dest_id'] + geo_cols ].copy()
y_r = routes['exists'].copy()

X_r.loc[:, geo_cols] = preprocessing.StandardScaler().fit_transform( X_r[geo_cols] )

In [33]:
X_r.head()

,origin_id,dest_id,airport_points_lat_ori,airport_points_lon_ori,airport_points_lat_dest,airport_points_lon_dest
index,,,,,,
45689,21,3,-0.054206,1.048145,1.439869,1.242656
131899,189,156,1.838833,-3.885052,1.770004,-2.973792
900326,208,144,-0.547689,0.134384,-0.846705,0.795188
185477,179,25,-0.264626,-0.799974,-0.080429,0.991109
354011,209,114,-1.710272,-0.334745,-1.216880,0.049915


In [34]:
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_r, y_r, test_size=0.2, random_state=42)

In [35]:
print('Dataset sizes:')
print('    Train:      {}'.format(X_train_r.shape))
print('    Test:       {}'.format(X_test_r.shape))

Dataset sizes:
    Train:      (723742, 6)
    Test:       (180936, 6)


In [36]:
X_train_r.head()

,origin_id,dest_id,airport_points_lat_ori,airport_points_lon_ori,airport_points_lat_dest,airport_points_lon_dest
index,,,,,,
726856,119,43,-0.141433,-0.209571,1.782907,1.500410
626279,66,171,-1.310651,0.073808,0.873058,-0.028664
521257,25,58,-0.046121,0.990981,2.019833,1.566576
727081,119,66,-0.141433,-0.209571,-1.326522,0.109874
467539,0,213,-0.137685,-1.128450,-0.888037,-1.470259


In [37]:
class BaseEmbeddingModel(metaclass=ABCMeta):
    
    def compile(self, optimizer, loss, *args, **kwargs):
        self.model.compile(optimizer, loss)
        
    def summary(self):
        return self.model.summary()
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def predict(self, X):
        return self.model.predict(X)
        
    @abstractmethod
    def _build_model(self):
        pass

In [38]:

N_UNIQUE_AIRPORTS = len(unique_airports)


class DotProductEmbeddingModel(BaseEmbeddingModel):
    
    def __init__(self, embedding_size, w, n_unique_airports=N_UNIQUE_AIRPORTS):
        
        self.n_unique_airports = n_unique_airports
        self.embedding_size = embedding_size
        self.w = w
        self.model = self._build_model()
        
    def _build_model(self):
        # inputs
        origin = Input(shape=(1,), name='origin')
        dest = Input(shape=(1,), name='dest')
        origin_geo = Input(shape=(2,), name='origin_geo')
        dest_geo = Input(shape=(2,), name='dest_geo')
    
        # embeddings
        origin_embedding = Embedding(self.n_unique_airports, output_dim=self.embedding_size, 
                                     embeddings_regularizer=l2(self.w), name='origin_embedding')(origin)
        dest_embedding = Embedding(self.n_unique_airports, output_dim=self.embedding_size, embeddings_regularizer=l2(self.w))(dest)

        # dot product
        dot_product = dot([origin_embedding, dest_embedding], axes=2)
        dot_product = Flatten()(dot_product)
        dot_product = concatenate([dot_product, origin_geo, dest_geo], axis=1)

        # dense layers
        tanh = Dense(10, activation='tanh')(dot_product)
        tanh = BatchNormalization()(tanh)

        # output
        exists = Dense(1, activation='sigmoid')(tanh)
        
        return Model(inputs=[origin, dest, origin_geo, dest_geo], outputs=[exists])


In [39]:
#N_UNIQUE_AIRPORTS = len(unique_airports)

In [40]:
dp_model = DotProductEmbeddingModel(embedding_size=10, w=0.001)

In [41]:
dp_model.compile(optimizer=Adam(lr=0.01), loss='binary_crossentropy', metrics=['accuracy'])
#SVG(model_to_dot(dp_model.model).create(prog='dot', format='svg'))

In [42]:
X_train_r.head()

,origin_id,dest_id,airport_points_lat_ori,airport_points_lon_ori,airport_points_lat_dest,airport_points_lon_dest
index,,,,,,
726856,119,43,-0.141433,-0.209571,1.782907,1.500410
626279,66,171,-1.310651,0.073808,0.873058,-0.028664
521257,25,58,-0.046121,0.990981,2.019833,1.566576
727081,119,66,-0.141433,-0.209571,-1.326522,0.109874
467539,0,213,-0.137685,-1.128450,-0.888037,-1.470259


In [44]:
dp_model_fit = dp_model.fit(
    x=[
        X_train_r['origin_id'], 
        X_train_r['dest_id'],
        X_train_r[['airport_points_lat_ori', 'airport_points_lon_ori']].as_matrix(),
        X_train_r[['airport_points_lat_dest', 'airport_points_lon_dest']].as_matrix(),
    ],
    y=y_train_r,
    batch_size=256, 
    epochs=100,
    validation_split=0.2
)

plot_model_fit(dp_model_fit)

Train on 578993 samples, validate on 144749 samples
Epoch 1/100
578993/578993 [==============================] - 16s 27us/step - loss: 0.6403 - val_loss: 0.6265
Epoch 2/100
578993/578993 [==============================] - 15s 26us/step - loss: 0.6257 - val_loss: 0.6233
Epoch 3/100
578993/578993 [==============================] - 15s 26us/step - loss: 0.6236 - val_loss: 0.6203
Epoch 4/100
578993/578993 [==============================] - 15s 26us/step - loss: 0.6224 - val_loss: 0.6221
Epoch 5/100
578993/578993 [==============================] - 15s 26us/step - loss: 0.6217 - val_loss: 0.6234
Epoch 6/100
578993/578993 [==============================] - 15s 26us/step - loss: 0.6211 - val_loss: 0.6186
Epoch 7/100
578993/578993 [==============================] - 15s 26us/step - loss: 0.6209 - val_loss: 0.6221
Epoch 8/100
578993/578993 [==============================] - 15s 26us/step - loss: 0.6211 - val_loss: 0.6225
Epoch 9/100
578993/578993 [==============================] - 15s 26us/step -

578993/578993 [==============================] - 15s 26us/step - loss: 0.6199 - val_loss: 0.6176
Epoch 76/100
578993/578993 [==============================] - 15s 26us/step - loss: 0.6198 - val_loss: 0.6154
Epoch 77/100
578993/578993 [==============================] - 15s 26us/step - loss: 0.6199 - val_loss: 0.6182
Epoch 78/100
578993/578993 [==============================] - 15s 26us/step - loss: 0.6199 - val_loss: 0.6210
Epoch 79/100
578993/578993 [==============================] - 15s 26us/step - loss: 0.6201 - val_loss: 0.6205
Epoch 80/100
578993/578993 [==============================] - 15s 26us/step - loss: 0.6199 - val_loss: 0.6179
Epoch 81/100
578993/578993 [==============================] - 15s 26us/step - loss: 0.6198 - val_loss: 0.6182
Epoch 82/100
578993/578993 [==============================] - 15s 26us/step - loss: 0.6202 - val_loss: 0.6181
Epoch 83/100
578993/578993 [==============================] - 15s 26us/step - loss: 0.6202 - val_loss: 0.6184
Epoch 84/100
578993/578

NameError: name 'plot_model_fit' is not defined

In [ ]:
class VariationalLayer(KerasLayer):

    def __init__(self, output_dim, epsilon_std=1.):

        self.output_dim = output_dim
        self.epsilon_std = epsilon_std
        super().__init__()
        
    def build(self, input_shape):
        self.z_mean_weights = self.add_weight(
            shape=(input_shape[1], self.output_dim),
            initializer='glorot_normal',
            trainable=True,
            name = 'z_mean_weights'
        )
        self.z_mean_bias = self.add_weight(
            shape=(self.output_dim,),
            initializer='zero',
            trainable=True,
            name = 'z_mean_bias'
        )
        self.z_log_var_weights = self.add_weight(
            shape=(input_shape[1], self.output_dim),
            initializer='glorot_normal',
            trainable=True,
            name = 'z_log_var_weights'
        )
        self.z_log_var_bias = self.add_weight(
            shape=(self.output_dim,),
            initializer='zero',
            trainable=True,
            name = 'z_log_var_bias'
        )
        super().build(input_shape)
        
    def call(self, x):
        z_mean = K.dot(x, self.z_mean_weights) + self.z_mean_bias
        z_log_var = K.dot(x, self.z_log_var_weights) + self.z_log_var_bias
        epsilon = K.random_normal(
            shape=K.shape(z_log_var),
            mean=0.,
            stddev=self.epsilon_std
        )
        
        kl_loss_numerator = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
        self.kl_loss = -0.5 * K.sum(kl_loss_numerator, axis=-1)
        return z_mean + K.exp(z_log_var / 2) * epsilon
    
    def loss(self, x, x_decoded):
        return binary_crossentropy(x, x_decoded) + self.kl_loss
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [ ]:
class VariationalAutoEncoderEmbeddingModel(BaseEmbeddingModel):
    
    def __init__(self, embedding_size: int, dense_layer_size: int, w, n_unique_airports=N_UNIQUE_AIRPORTS):
    
        self.embedding_size = embedding_size
        self.dense_layer_size = dense_layer_size
        self.w = w
        self.n_unique_airports = n_unique_airports
        self.variational_layer = VariationalLayer(embedding_size)
        self.model = self._build_model()
        
    def _build_model(self):
        # encoder
        origin = Input(shape=(self.n_unique_airports,), name='origin')
        origin_geo = Input(shape=(2,), name='origin_geo')
        dense = concatenate([origin, origin_geo])
        dense = Dense(self.dense_layer_size, activation='tanh', kernel_regularizer=l2(self.w))(dense)
        dense = BatchNormalization()(dense)
        variational_output = self.variational_layer(dense)

        encoder = Model([origin, origin_geo], variational_output, name='encoder')

        # decoder
        latent_vars = Input(shape=(self.embedding_size,))
        dense = Dense(self.dense_layer_size, activation='tanh', kernel_regularizer=l2(self.w))(latent_vars)
        dense = Dense(self.dense_layer_size, activation='tanh', kernel_regularizer=l2(self.w))(dense)
        dense = BatchNormalization()(dense)
        dest = Dense(self.n_unique_airports, activation='softmax', name='dest', kernel_regularizer=l2(self.w))(dense)
        dest_geo = Dense(2, activation='linear', name='dest_geo')(dense)

        decoder = Model(latent_vars, [dest, dest_geo], name='decoder')

        # end-to-end
        encoder_decoder = Model([origin, origin_geo], decoder(encoder([origin, origin_geo])))
        return encoder_decoder

In [ ]:
vae_model = VariationalAutoEncoderEmbeddingModel(embedding_size=10, dense_layer_size=20, w=0)
vae_model.compile(optimizer=Adam(lr=1), loss=[vae_model.variational_layer.loss, 'binary_crossentropy'], 
                  loss_weights=[1.0, 0.2])
SVG(model_to_dot(vae_model.model).create(prog='dot', format='svg'))

In [ ]:
dp_model_fit = dp_model.fit(
    x=[
        X_train_r['origin_id'], 
        X_train_r['dest_id'],
        X_train_r[['airport_points_lat_ori', 'airport_points_lon_ori']].values,
        X_train_r[['airport_points_lat_dest', 'airport_points_lon_dest']].values,
    ],
    y=y_train_r,
    batch_size=512, 
    epochs=100,
    validation_split=0.2
)

plot_model_fit(dp_model_fit)

In [ ]:
X_train_r[['airport_points_lat_ori', 'airport_points_lon_ori']].values

In [ ]:
def get_vae_embeddings(vae_model, unique_origins=unique_origins):
    '''Returns the origin airport embeddings from the variational autoencoder embedding model,
    *aligned with the index of `unique_origins`*.
    '''
    encoder_inputs = [
        one_hot_airports[ unique_origins['origin_id'] ],
        unique_origins[['latitude', 'longitude']].as_matrix()
    ]
    return vae_model.model.get_layer('encoder').predict(encoder_inputs)


unique_origins['embedding'] = get_vae_embeddings(vae_model)
plot_embeddings_on_world_map(unique_origins, output_path='../figures/vae_model_map.html')

In [ ]:
def get_dp_embeddings(dp_model, unique_origins=unique_origins):
    '''Returns the origin airport embeddings from the dot-product embedding model,
    *aligned with the index of `unique_origins`*.
    '''
    origin_embeddings = dp_model.model.get_layer(name='origin_embedding').get_weights()[0]
    return origin_embeddings[ unique_origins['origin_id'].values ]

In [ ]:
# dot product embedding
EMBEDDING_SIZE = 20


dp_model = DotProductEmbeddingModel(embedding_size=EMBEDDING_SIZE, w=.0001)
dp_model.compile(optimizer=Adam(lr=LEARNING_RATE), loss='binary_crossentropy')

dp_model_fit = dp_model.fit(
    x=[
        X_r['origin_id'], 
        X_r['dest_id'],
        X_r[['origin_latitude', 'origin_longitude']].as_matrix(),
        X_r[['dest_latitude', 'dest_longitude']].as_matrix(),
    ],
    y=y_r,
    batch_size=256, 
    epochs=5,
    verbose=0,
    callbacks=[TQDMNotebookCallback(leave_outer=False)]
)

In [ ]:
# variational autoencoder embedding
vae_model = VariationalAutoEncoderEmbeddingModel(embedding_size=EMBEDDING_SIZE, dense_layer_size=30, w=.003)
vae_model.compile(optimizer=Adam(lr=LEARNING_RATE), loss=[vae_model.variational_layer.loss, 'mean_squared_logarithmic_error'], 
                  loss_weights=[1., .2])

X_r_origin = one_hot_airports[ X_r['origin_id'] ]
X_r_dest   = one_hot_airports[ X_r['dest_id']   ]

vae_model_fit = vae_model.fit(
    x=[
        X_r_origin,
        X_r[['origin_latitude', 'origin_longitude']].as_matrix()
    ],
    y=[
        X_r_dest,
        X_r[['dest_latitude', 'dest_longitude']].as_matrix()
    ],
    batch_size=1024, 
    epochs=5,
    verbose=0,
    callbacks=[TQDMNotebookCallback(leave_outer=False)]
)